# Part 1

## Is 'Academy Dinosaur' available for rent from Store 1?

In [ ]:
# join
SELECT i.*
FROM inventory i
JOIN film f
ON i.film_id=f.film_id
WHERE f.title = 'Academy Dinosaur'
AND i.store_id = 1;

In [ ]:
# sub query
SELECT *
FROM inventory
WHERE film_id IN (
  SELECT film_id
  FROM film
  WHERE title = 'Academy Dinosaur'
)
AND store_id = 1;

## Step 1: which copies are at Store 1?

In [ ]:
SELECT i.inventory_id, i.film_id, f.title, COUNT(i.film_id) AS Num_copies
FROM inventory i
LEFT JOIN film f
ON i.film_id=f.film_id
WHERE i.store_id = 1
GROUP BY i.film_id;

## Step 2: pick an inventory_id to rent:

WITH temp AS (
  SELECT i.inventory_id, i.film_id, f.title, COUNT(i.film_id) AS Num_copies
  FROM inventory i
  LEFT JOIN film f
  ON i.film_id=f.film_id
  WHERE i.store_id = 1
  GROUP BY i.film_id)
   
 SELECT *
 FROM temp
 WHERE inventory_id = 1;

## Insert a record to represent Mary Smith renting 'Academy Dinosaur' from Mike Hillyer at Store 1 today .

In [ ]:
# Create table
CREATE TEMPORARY TABLE temp_rental AS (
   SELECT * FROM rental
);

# Set variable values
SET @new_rental_id = ((SELECT MAX(rental_id)
                       FROM rental)
                       + 1);

SET @new_rental_duration = (SELECT rental_duration 
                            FROM film
                            WHERE title = 'Academy Dinosaur');

SET @return_time = (SELECT DATE_ADD(CURRENT_TIMESTAMP, INTERVAL @new_rental_duration DAY));

SET @new_rental_customer_id = (SELECT customer_id 
                               FROM customer
                               WHERE first_name = 'Mary'
                               AND last_name = 'Smith');

SET @new_rental_staff_id = (SELECT staff_id 
                            FROM staff
                            WHERE first_name = 'Mike'
                            AND last_name = 'Hillyer');

# Add a new record
INSERT INTO temp_rental (rental_id, rental_date, inventory_id, 
                         customer_id, return_date, staff_id, last_update)
VALUES (@new_rental_id, CURRENT_TIMESTAMP, 1,
        @new_rental_customer_id, @return_time, @new_rental_staff_id, CURRENT_TIMESTAMP);

#Show the latest record rental_id
SELECT * 
FROM temp_rental
ORDER BY 1 DESC
LIMIT 1;

### When is 'Academy Dinosaur' due?

In [ ]:
SELECT DATE_ADD(CURRENT_TIMESTAMP, INTERVAL @new_rental_duration DAY) AS Due_day;

#### Step 1: what is the rental duration?

In [ ]:
SELECT film_id, title, rental_duration 
FROM film
WHERE title = 'Academy Dinosaur';

#### Step 2: Which rental are we referring to -- the last one.

In [ ]:
SELECT * 
FROM temp_rental
ORDER BY 1 DESC
LIMIT 1;

#### Step 3: add the rental duration to the rental date.

In [ ]:
SET @return_time = (SELECT DATE_ADD(@today_time, INTERVAL @new_rental_duration DAY));

## What is that average length of all the films in the sakila DB?

In [ ]:
SELECT AVG(length) AS Avg_length
FROM film;

## What is the average length of films by category?

In [ ]:
SELECT fc.category_id, AVG(f.length)
FROM film f
JOIN film_category fc
ON f.film_id=fc.film_id
GROUP BY fc.category_id;

## Which film categories are long? Long = lengh is longer than the average film length

In [ ]:
SET @avg_length_all = (SELECT AVG(length)
                       FROM film);

SELECT fc.category_id, AVG(f.length) AS avg_length
FROM film f
JOIN film_category fc
ON f.film_id=fc.film_id
GROUP BY fc.category_id
HAVING avg_length > @avg_length_all;

# Part 2

## 1a. Display the first and last names of all actors from the table actor.

In [ ]:
SELECT actor_id, first_name, last_name 
FROM actor;

## 1b. Display the first and last name of each actor in a single column in upper case letters. Name the column Actor Name.

In [ ]:
CREATE TEMPORARY TABLE temp_actor AS (
   SELECT actor_id, first_name, last_name 
   FROM actor
);

ALTER TABLE temp_actor
ADD actor_name VARCHAR(40) GENERATED ALWAYS AS (CONCAT(UPPER(first_name),' ', UPPER(last_name)));

SELECT * FROM temp_actor;

## 2a. You need to find the ID number, first name, and last name of an actor, of whom you know only the first name, "Joe." What is one query would you use to obtain this information?

In [ ]:
SELECT * 
FROM actor
WHERE first_name = 'Joe';

## 2b. Find all actors whose last name contain the letters GEN:

In [ ]:
SELECT * 
FROM actor
WHERE last_name LIKE '%GEN%';

## 2c. Find all actors whose last names contain the letters LI. This time, order the rows by last name and first name, in that order:

In [ ]:
SELECT * 
FROM actor
WHERE last_name LIKE '%LI%'
ORDER BY 3, 2;

## 2d. Using IN, display the country_id and country columns of the following countries: Afghanistan, Bangladesh, and China:

In [ ]:
SELECT country_id, country 
FROM country
WHERE country IN ('Afghanistan', 'Bangladesh', 'China');

## 3a. Add a middle_name column to the table actor. Position it between first_name and last_name. Hint: you will need to specify the data type.

In [ ]:
CREATE TEMPORARY TABLE temp_actor AS (
   SELECT actor_id, first_name, last_name 
   FROM actor
);

ALTER TABLE temp_actor
ADD middle_name VARCHAR(40) AFTER first_name;

SELECT * FROM temp_actor;

## 3b. You realize that some of these actors have tremendously long last names. Change the data type of the middle_name column to blobs.

In [ ]:
ALTER TABLE temp_actor
MODIFY COLUMN middle_name BLOB;

## 3c. Now delete the middle_name column.

In [ ]:
ALTER TABLE temp_actor
DROP COLUMN middle_name;

## 4a. List the last names of actors, as well as how many actors have that last name.

In [ ]:
SELECT last_name, COUNT(*) AS Num_actor
FROM actor
GROUP BY last_name
ORDER BY 2 DESC;

## 4b. List last names of actors and the number of actors who have that last name, but only for names that are shared by at least two actors

In [ ]:
SELECT last_name, COUNT(*) AS Num_actor
FROM actor
GROUP BY last_name
HAVING Num_actor >= 2
ORDER BY 2 DESC;

## 4c. Oh, no! The actor HARPO WILLIAMS was accidentally entered in the actor table as GROUCHO WILLIAMS, the name of Harpo's second cousin's husband's yoga teacher. Write a query to fix the record.

In [ ]:
CREATE TEMPORARY TABLE temp_actor AS (
   SELECT actor_id, first_name, last_name 
   FROM actor
);

SET @target_id = (SELECT actor_id 
                  FROM temp_actor
                  WHERE first_name = 'GROUCHO'
                  AND last_name = 'WILLIAMS');

UPDATE temp_actor
SET first_name = 'HARPO'
WHERE actor_id = @target_id;

## 4d. Perhaps we were too hasty in changing GROUCHO to HARPO. It turns out that GROUCHO was the correct name after all! In a single query, if the first name of the actor is currently HARPO, change it to GROUCHO. Otherwise, change the first name to MUCHO GROUCHO, as that is exactly what the actor will be with the grievous error. BE CAREFUL NOT TO CHANGE THE FIRST NAME OF EVERY ACTOR TO MUCHO GROUCHO, HOWEVER! (Hint: update the record using a unique identifier.)

In [ ]:
# Check unique or not
SELECT first_name, COUNT(*)
FROM temp_actor
WHERE first_name IN ('HARPO', 'GROUCHO');

UPDATE temp_actor
SET first_name = CASE WHEN first_name = 'HARPO' THEN 'GROUCHO'
                      WHEN first_name = 'GROUCHO' THEN 'MUCHO GROUCHO'
                      ELSE first_name
                 END;
                 
SELECT * FROM temp_actor
WHERE first_name LIKE ('%GROUCHO%');

## 5a. You cannot locate the schema of the address table. Which query would you use to re-create it?

In [ ]:
CREATE TABLE address2 (address_id INT NOT NULL PRIMARY KEY,
                       address VARCHAR(50) NOT NULL,
                       adress2 VARCHAR(50),
                       district VARCHAR(20) NOT NULL,
                       city_id INT,
                       postal_code INT,
                       phone INT,
                       location BLOB NOT NULL,
                       last_update DATETIME
                       );

## 6a. Use JOIN to display the first and last names, as well as the address, of each staff member. Use the tables staff and address:

In [ ]:
SELECT s.staff_id, s.first_name, s.last_name, a.address_id, a.address
FROM staff s
JOIN address a
ON s.address_id=a.address_id;

## 6b. Use JOIN to display the total amount rung up by each staff member in August of 2005. Use tables staff and payment.

In [ ]:
SELECT s.staff_id, s.first_name, s.last_name, SUM(p.amount) AS Total_amount
FROM payment p
JOIN staff s
ON p.staff_id=s.staff_id
WHERE p.payment_date LIKE '2005-08%'
GROUP BY p.staff_id;

## 6c. List each film and the number of actors who are listed for that film. Use tables film_actor and film. Use inner join.

In [ ]:
SELECT f.film_id, f.title, COUNT(DISTINCT fa.actor_id) AS Num_actor
FROM film f
JOIN film_actor fa
ON f.film_id=fa.film_id
GROUP BY fa.film_id;

## 6d. How many copies of the film Hunchback Impossible exist in the inventory system?

In [ ]:
SELECT f.film_id, f.title, COUNT(*) AS Num_copies
FROM inventory i
LEFT JOIN film f
ON i.film_id=f.film_id
WHERE f.title = 'Hunchback Impossible'
GROUP BY f.title;

## 7a. The music of Queen and Kris Kristofferson have seen an unlikely resurgence. As an unintended consequence, films starting with the letters K and Q have also soared in popularity. Use subqueries to display the titles of movies starting with the letters K and Q whose language is English.

In [ ]:
WITH tl AS (SELECT * FROM language
            WHERE name = 'English')

SELECT f.film_id, f.title, tl.name AS language_name
FROM film f
LEFT JOIN tl
ON f.language_id=tl.language_id
WHERE f.title LIKE 'K%' OR 'Q%'

# OR
SELECT f.film_id, f.title, la.name AS language_name
FROM film f
LEFT JOIN language la
ON f.language_id=la.language_id
WHERE f.title LIKE 'K%' OR 'Q%' 
AND la.name = 'English';

## 7b. Use subqueries to display all actors who appear in the film Alone Trip.

In [ ]:
SELECT fa.film_id, f.title, fa.actor_id, CONCAT(a.first_name,' ', a.last_name) AS actor_name
FROM film_actor fa
JOIN actor a
  ON fa.actor_id=a.actor_id
JOIN film f
  ON fa.film_id=f.film_id
WHERE f.title='Alone Trip';

## 7c. You want to run an email marketing campaign in Canada, for which you will need the names and email addresses of all Canadian customers. Use joins to retrieve this information.

In [ ]:
SELECT cu.customer_id, CONCAT(cu.first_name, ' ', cu.last_name) AS Customer_name, cu.email, co.country
FROM customer cu
JOIN address ad 
  ON cu.address_id=ad.address_id
JOIN city 
  ON ad.city_id=city.city_id
JOIN country co 
  ON city.country_id=co.country_id
WHERE co.country='Canada'
ORDER BY 1;

## 7d. Sales have been lagging among young families, and you wish to target all family movies for a promotion. Identify all movies categorized as famiy films.

In [ ]:
SELECT f.film_id, f.title, c.name AS film_cat_name 
FROM film f
JOIN film_category fc 
  ON f.film_id=fc.film_id
JOIN category c 
  ON fc.category_id=c.category_id
WHERE c.name = 'family';

## 7e. Display the most frequently rented movies in descending order.

In [ ]:
SELECT f.film_id, f.title, COUNT(DISTINCT r.rental_id) AS Num_rented
FROM rental r
JOIN inventory inv 
  ON r.inventory_id=inv.inventory_id
JOIN film f
  ON inv.film_id=f.film_id
GROUP BY f.film_id
ORDER BY 3 DESC;

## 7f. Write a query to display how much business, in dollars, each store brought in.

In [ ]:
SELECT s.store_id, SUM(p.amount) AS Total_amount
FROM payment p
JOIN staff s
  ON p.staff_id=s.staff_id
GROUP BY s.store_id;

## 7g. Write a query to display for each store its store ID, city, and country.

In [ ]:
SELECT s.store_id, ci.city, co.country
FROM store s
JOIN address a
  ON s.address_id=a.address_id
JOIN city ci
  ON a.city_id=ci.city_id
JOIN country co
  ON ci.country_id=co.country_id;

## 7h. List the top five genres in gross revenue in descending order. (Hint: you may need to use the following tables: category, film_category, inventory, payment, and rental.)

In [ ]:
SELECT c.category_id, c.name AS Cat_name, SUM(p.amount) AS Total_amount
FROM payment p
JOIN rental r
  ON p.rental_id=r.rental_id
JOIN inventory i
  ON r.inventory_id=i.inventory_id
JOIN film_category fc
  ON i.film_id=fc.film_id
JOIN category c
  ON fc.category_id=c.category_id
GROUP BY c.category_id
ORDER BY 3 DESC
LIMIT 5;

## 8a. In your new role as an executive, you would like to have an easy way of viewing the Top five genres by gross revenue. Use the solution from the problem above to create a view. If you haven't solved 7h, you can substitute another query to create a view.

In [ ]:
CREATE VIEW top_5_genres AS
SELECT c.category_id, c.name AS Cat_name, SUM(p.amount) AS Total_amount
FROM payment p
JOIN rental r
  ON p.rental_id=r.rental_id
JOIN inventory i
  ON r.inventory_id=i.inventory_id
JOIN film_category fc
  ON i.film_id=fc.film_id
JOIN category c
  ON fc.category_id=c.category_id
GROUP BY c.category_id
ORDER BY 3 DESC
LIMIT 5;

## 8b. How would you display the view that you created in 8a?

In [ ]:
SELECT * FROM top_5_genres;

## 8c. You find that you no longer need the view top_five_genres. Write a query to delete it.

In [ ]:
DROP VIEW top_5_genres;